In [124]:
from lstm_creator import createLstmModelFromDatasets, scaleData, preprocessData
from predict_next_value import predict_next, load_x_from_file

import matplotlib.pyplot as plt
%matplotlib inline

In [125]:
# use the last 90 minutes to predict next value
lookback_length = 20

# data is stores from every 15 seconds, but we only want every 4th item
sample_every_x = 3

paths = ["../../../data/dump.csv"]
model = createLstmModelFromDatasets(paths, lookback_length, sample_every_x, epochs=60)

# Uncomment this line to save the model on completion
# model.save('../../../models/dump_model_1')

Epoch 1/60
57/57 [==============================] - 3s 11ms/step - loss: 0.0757
Epoch 2/60
57/57 [==============================] - 1s 11ms/step - loss: 0.0072
Epoch 3/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0073
Epoch 4/60
57/57 [==============================] - 1s 11ms/step - loss: 0.0065
Epoch 5/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0064
Epoch 6/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0062
Epoch 7/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0063
Epoch 8/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0066
Epoch 9/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0058
Epoch 10/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0061
Epoch 11/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0053
Epoch 12/60
57/57 [==============================] - 1s 10ms/step - loss: 0.0051
Epoch 13/60
57/57 [==================

# Load model

In [126]:
from tensorflow.keras import models
from lstm_creator import createLstmModelFromDatasets, scaleData, preprocessData
import numpy as np

# Uncomment this line to load the model from a file for testing
# model = models.load_model('../../../models/dump_model_1')

# Testing

In [127]:
def evaluate_model(real_data, predicted_data, slope_to_buy_at, inherent_loss=2):
    real_data = real_data.reshape(len(real_data))
    predicted_data = predicted_data.reshape(len(predicted_data))

    real_diff = np.diff(real_data)
    predicted_diff = np.diff(predicted_data)

    correct_slopes = 0
    profit = 0
    profits = 0
    losses = 0
    for i in range(len(real_data)-1):
        if np.sign(real_diff[i]) == np.sign(predicted_diff[i]):
            correct_slopes = correct_slopes + 1
            
            # If we have a positive slope calculate profit
            if real_diff[i] > slope_to_buy_at:
                # we subtract inherent_loss due to the limit market mechanics
                revenue = (real_data[i+1] - real_data[i]) - inherent_loss
                if revenue > 0:
                    # print(f"Found a profit where current value is {real_data[i+1]} last was {real_data[i]} net {revenue}")
                    profits = profits + 1
                    profit = profit + revenue

        else:
            # We guessed wrong
            if predicted_diff[i] > slope_to_buy_at:
                # we would have bought
                revenue = (real_data[i+1] - real_data[i]) - inherent_loss
                # print(f"Selling at a loss of {revenue}")
                losses = losses + 1
                profit = profit + revenue

    return (correct_slopes, profit, profits, losses)

def eval_model_on_dataset(actual, prediction, slope_to_buy_at, subsampling, inherent_loss):
    # Determine the number of cases in which we predicted a correct increase
    correct_slopes, profit, profits, losses = evaluate_model(actual, prediction, slope_to_buy_at, inherent_loss)

    # print(f"Found {correct_slopes} out of {len(actual)-1}")
    precent_success = (correct_slopes/(len(actual)-1)) * 100
    # print(f"{precent_success}%")
    # print("Profit:", profit)
    return precent_success, profit, profits, losses

In [128]:
def testModel(model, length, slope_to_buy_at, path_to_testing_dataset, quiet=False):

    datasets, scaler = scaleData([path_to_testing_dataset], 3)

    hist, actual = preprocessData(datasets[0], length)

    pred = model.predict(hist)

    pred_transformed = scaler.inverse_transform(pred)
    actual_transformed = scaler.inverse_transform(actual)

    if not quiet:
        plt.figure(figsize=(12,8))
        plt.plot(actual_transformed, color='blue', label='Real')
        plt.plot(pred_transformed, color='red', label='Prediction')
        plt.title('ETH Price Prediction')
        plt.legend()
        plt.show()

    return eval_model_on_dataset(actual=actual_transformed, prediction=pred_transformed, slope_to_buy_at=slope_to_buy_at, subsampling=1, inherent_loss=1.5)

In [129]:
overall = (0, 0, 0, 0)
lookback_length = 20
slope_to_buy_at = 1.75

result1 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test6.csv", quiet=True)
accuracy, profit, profits, losses = result1
print(f"Accuracy: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

result2 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test8.csv", quiet=True)
accuracy, profit, profits, losses = result2
print(f"Accuracy: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

result3 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test10.csv", quiet=True)
accuracy, profit, profits, losses = result3
print(f"Accuracy: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

overall = [sum(x) for x in zip(result1,overall)]
overall = [sum(x) for x in zip(result2,overall)]
overall = [sum(x) for x in zip(result2,overall)]
print("-----------------------------------")
print(f"Accuracy: {overall[0]/3:.2f}% Profit: ${overall[1]:.2f} (Profits/Losses: {overall[2]}/{overall[3]})")



Accuracy: 47.52% Profit: $108.10 (Profits/Losses: 62/1)
Accuracy: 48.34% Profit: $94.32 (Profits/Losses: 66/0)
Accuracy: 49.33% Profit: $115.85 (Profits/Losses: 72/1)
-----------------------------------
Accuracy: 48.07% Profit: $296.74 (Profits/Losses: 194/1)
